In [220]:
import numpy as np
import math
import torch

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [3]:
import pandas as pd

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import importlib

In [6]:
import lib
importlib.reload(lib)

<module 'lib' from 'c:\\Users\\dongq\\OneDrive\\Desktop\\New results\\Portfolio Allocation\\Learning-the-Optimal-Solution-Path\\lib\\__init__.py'>

In [7]:
from lib.fast_tensor_data_loader import FastTensorDataLoader
from lib.lsp.basis_generator import bivariate_legendre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.loss_fn_lsp import allocation_cost
from lib.lsp.learn_solution_path import learn_solution_path

In [222]:
# file path for Colab. May need to change this
# X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
# y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')
decomp_cov_df = pd.read_csv('decomp_pert_cov.csv')
decomp_cov = np.array(decomp_cov_df)

In [187]:
# # uncomment and run this cell to replace the covariance matrix with eye

# identity_matrix = np.eye(len(decomp_cov))
# decomp_cov = identity_matrix

In [223]:
first_column = decomp_cov[:, 0]
first_column

array([4.98868198, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [224]:
n = decomp_cov.shape[1]
# first_column.repeat(1, n - 1)
repeated_matrix = np.tile(first_column.reshape(-1, 1), (1, n-1))
repeated_matrix

array([[4.98868198, 4.98868198, 4.98868198, 4.98868198, 4.98868198,
        4.98868198, 4.98868198, 4.98868198, 4.98868198],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],


In [225]:
decomp_cov[:, 1:]

array([[ 3.60083657e+00,  3.13259052e+00,  3.63945434e+00,
         2.57533640e+00,  2.69706673e+00,  2.48434213e+00,
         2.10188073e+00,  2.15928287e+00,  3.01761728e+00],
       [ 1.05518856e+01,  2.01024062e+00,  2.20799777e+00,
         2.98069420e+00,  1.21427926e+00,  2.51026590e+00,
         1.29594640e+00,  3.92258404e-01,  2.36064741e+00],
       [ 0.00000000e+00,  4.66569521e+00,  2.67910121e+00,
         1.52890546e+00,  1.65585326e+00,  1.39125907e+00,
         1.48903773e+00,  1.00267743e+00,  2.21191348e+00],
       [ 0.00000000e+00,  0.00000000e+00,  7.86096826e+00,
        -7.43108406e-02,  9.51614270e-01,  4.71482592e-02,
         2.43258784e-01,  2.41179895e-01,  1.22688033e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.69328792e+00,  7.07176102e-01,  1.42678877e+00,
         9.86191341e-01,  4.09449522e-01,  7.31587339e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  4.56375352e+00,  4.685418

In [226]:
# the risk term will be constant when we take hessian
const = decomp_cov[:, 1:] + repeated_matrix
2 * const.T @ const

array([[370.24423815, 181.93929975, 194.82015387, 192.84644092,
        157.65963395, 181.35543555, 149.15839605, 131.07328487,
        187.35907487],
       [181.93929975, 183.52969231, 174.01984542, 149.10954777,
        145.1695195 , 144.45578878, 134.27390433, 127.03458277,
        160.1738429 ],
       [194.82015387, 174.01984542, 296.58479158, 150.71337035,
        161.82324308, 148.23777405, 139.88264141, 134.24380518,
        179.72430615],
       [192.84644092, 149.10954777, 150.71337035, 180.93787494,
        135.06889709, 145.65671427, 128.76593514, 117.34656204,
        148.6406978 ],
       [157.65963395, 145.1695195 , 161.82324308, 135.06889709,
        171.04114263, 131.9596749 , 123.63480207, 118.0604902 ,
        146.45776801],
       [181.35543555, 144.45578878, 148.23777405, 145.65671427,
        131.9596749 , 166.22712873, 126.31323531, 115.86082759,
        144.12080066],
       [149.15839605, 134.27390433, 139.88264141, 128.76593514,
        123.63480207, 126.3132

In [192]:
# Read the CSV file into a DataFrame
thetastar_df = pd.read_csv('thetastar_eye_001_20_20.csv')

# Display the DataFrame
thetastar_df

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,1.000000,0.313889,0.024895,0.010327,0.359538,0.004093,0.156354,0.023870,0.011280,0.072879
1,1.0,0.947368,0.307511,0.028591,0.011717,0.350753,0.005149,0.158274,0.027379,0.012766,0.078786
2,1.0,0.894737,0.300975,0.033236,0.013601,0.341810,0.006652,0.160042,0.031823,0.014768,0.084678
3,1.0,0.842105,0.292934,0.037685,0.014997,0.331362,0.007443,0.160314,0.036093,0.016315,0.089195
4,1.0,0.789474,0.284425,0.042627,0.016623,0.320445,0.008310,0.160121,0.040894,0.018132,0.093311
...,...,...,...,...,...,...,...,...,...,...,...
395,0.2,0.210526,0.294448,0.037558,0.024626,0.342211,0.017392,0.133336,0.036856,0.025693,0.064399
396,0.2,0.157895,0.260690,0.050944,0.032210,0.296382,0.022253,0.140166,0.049948,0.033740,0.083084
397,0.2,0.105263,0.217007,0.069793,0.045811,0.240593,0.030987,0.137564,0.068703,0.048033,0.098092
398,0.2,0.052632,0.162169,0.088336,0.069704,0.173616,0.051369,0.123829,0.087643,0.071858,0.104238


In [193]:
thetastar = thetastar_df.to_numpy()[:, 2:]
thetastar

array([[0.31388924, 0.02489488, 0.01032669, ..., 0.02386997, 0.01127952,
        0.07287895],
       [0.30751058, 0.02859099, 0.01171684, ..., 0.02737912, 0.01276584,
        0.07878579],
       [0.30097458, 0.03323609, 0.01360118, ..., 0.03182335, 0.0147679 ,
        0.08467773],
       ...,
       [0.21700656, 0.06979271, 0.04581115, ..., 0.06870284, 0.04803346,
        0.09809209],
       [0.16216861, 0.0883356 , 0.06970379, ..., 0.08764255, 0.07185751,
        0.10423835],
       [0.10002528, 0.09999228, 0.09998608, ..., 0.09999205, 0.09998675,
        0.09999831]])

In [194]:
hyper_params = thetastar_df.to_numpy()[:, :2]
hyper_params

array([[1.        , 1.        ],
       [1.        , 0.94736842],
       [1.        , 0.89473684],
       [1.        , 0.84210526],
       [1.        , 0.78947368],
       [1.        , 0.73684211],
       [1.        , 0.68421053],
       [1.        , 0.63157895],
       [1.        , 0.57894737],
       [1.        , 0.52631579],
       [1.        , 0.47368421],
       [1.        , 0.42105263],
       [1.        , 0.36842105],
       [1.        , 0.31578947],
       [1.        , 0.26315789],
       [1.        , 0.21052632],
       [1.        , 0.15789474],
       [1.        , 0.10526316],
       [1.        , 0.05263158],
       [1.        , 0.        ],
       [0.95789474, 1.        ],
       [0.95789474, 0.94736842],
       [0.95789474, 0.89473684],
       [0.95789474, 0.84210526],
       [0.95789474, 0.78947368],
       [0.95789474, 0.73684211],
       [0.95789474, 0.68421053],
       [0.95789474, 0.63157895],
       [0.95789474, 0.57894737],
       [0.95789474, 0.52631579],
       [0.

In [195]:
# returns hessian of the smoothing term
def hessian(theta, mu):
    n = len(theta)
    
    # Initialize Hessian matrix
    H = np.zeros((n, n))
    
    # First term: sum(sqrt(theta_i^2 + mu^2) - mu)
    for i in range(n):
        theta_i = theta[i]
        H[i, i] = mu**2 / (theta_i**2 + mu**2)**(3/2)

    # Second term: sqrt((1 - sum(theta))^2 + mu^2) - mu
    s_theta = 1 - np.sum(theta)
    s_theta_sq = s_theta**2
    
    # Compute second term contributions for all elements
    common_term = mu**2 / (s_theta_sq + mu**2)**(3/2)
    
    # Off-diagonal terms (same for all i, j)
    for i in range(n):
        for j in range(n):
            if i == j:
                # Diagonal elements get the extra term from the second part
                H[i, i] += common_term
            else:
                # Off-diagonal elements contribution from second part
                H[i, j] = common_term
    
    return H

In [259]:
mu = 10

In [197]:
# condition number using thetastar
n = len(thetastar[0])
sum_path = np.zeros((n, n))
for i in range(len(thetastar)):
    sum_path += hessian(thetastar[i], mu) + 2 * hyper_params[i][0] * const.T @ const
sim_exp_hessian_theta = sum_path/len(thetastar)
sim_exp_hessian_theta

array([[19.6782023 , 18.46425345, 18.46425345, 18.46425345, 18.46425345,
        18.46425345, 18.46425345, 18.46425345, 18.46425345],
       [18.46425345, 27.64360642, 18.46425345, 18.46425345, 18.46425345,
        18.46425345, 18.46425345, 18.46425345, 18.46425345],
       [18.46425345, 18.46425345, 37.45197352, 18.46425345, 18.46425345,
        18.46425345, 18.46425345, 18.46425345, 18.46425345],
       [18.46425345, 18.46425345, 18.46425345, 19.67642177, 18.46425345,
        18.46425345, 18.46425345, 18.46425345, 18.46425345],
       [18.46425345, 18.46425345, 18.46425345, 18.46425345, 53.10769799,
        18.46425345, 18.46425345, 18.46425345, 18.46425345],
       [18.46425345, 18.46425345, 18.46425345, 18.46425345, 18.46425345,
        20.33207272, 18.46425345, 18.46425345, 18.46425345],
       [18.46425345, 18.46425345, 18.46425345, 18.46425345, 18.46425345,
        18.46425345, 27.91993778, 18.46425345, 18.46425345],
       [18.46425345, 18.46425345, 18.46425345, 18.46425345, 18

In [198]:
eigs_hessian_theta = np.linalg.eigvals(sim_exp_hessian_theta)
eigs_hessian_theta

array([177.84572776,  31.55240275,  18.37252501,  14.74282903,
         1.21305768,   1.62224544,   3.44145445,   6.68502166,
         9.3200858 ])

In [199]:
# condition number for Hessian(\theta)
max(eigs_hessian_theta)/min(eigs_hessian_theta)

np.float64(146.60945718562382)

In [228]:
mean_df = pd.read_csv('mean.csv')
mean = np.array(mean_df).squeeze()

In [229]:
decomp_cov = torch.tensor(decomp_cov, dtype=torch.float32)
mean = torch.tensor(mean, dtype=torch.float32)

In [230]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )
# test data
test_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )

In [231]:
lam_max_2d = [1, 1]
lam_min_2d = [.2, 0]
input_dim = decomp_cov.shape[1] - 1

In [260]:
loss_fn = allocation_cost

In [233]:
phi_lam = bivariate_legendre

In [234]:
basis_dim = 3

In [235]:
init_lr = .5**9

In [236]:
max_epochs = 2000

In [237]:
def thresh_basis(basis_dim):
    return 1e-9

In [261]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min_2d, lam_max_2d, None, init_lr=init_lr,
                                                               diminish=True, gamma=0.97, q=1.5, k_0=5,
                                                               intercept=False, thresh_basis=None,
                                                               record_fctn=None,
                                                               record_frequency=-10, distribution='uniform',
                                                               device=device, trace_frequency=-10)

In [262]:
weight

tensor([[-6.0845e-02, -6.9534e-05,  8.3952e-05],
        [-2.1351e-02, -7.0854e-04,  3.5688e-04],
        [-2.9003e-02, -1.5149e-04,  1.5124e-04],
        [ 7.8327e-02,  4.0367e-03, -4.5203e-03],
        [ 1.0961e-01, -3.4330e-03,  3.7649e-03],
        [ 1.2461e-01,  1.1753e-03, -1.2900e-03],
        [ 2.2643e-01, -3.5120e-04,  6.1106e-04],
        [ 2.8010e-01, -3.5996e-04,  6.1186e-04],
        [ 3.6017e-02,  1.1931e-03, -1.3672e-03]])

In [263]:
model = Basis_TF_SGD(input_dim, basis_dim, phi_lam, weight, intercept=False).to(device)

In [253]:
model(hyper_params[0]).detach().numpy()

array([-0.0583592 , -0.00075523, -0.02277267,  0.06891014,  0.11006557,
        0.11951397,  0.22555134,  0.2793334 ,  0.02110789], dtype=float32)

In [254]:
bivariate_legendre(hyper_params[3], basis_dim).numpy()

array([1.       , 1.6052631, 1.6547784], dtype=float32)

In [255]:
def phi_matrix(a, n):
    # Create a n by (a*n) matrix of zeros
    matrix = np.zeros((n, n * len(a)))
    for i in range(n):
        # Assign 'a' to the appropriate positions
        matrix[i, i * len(a):(i+1) * len(a)] = a  

    return matrix

In [244]:
phi_matrix(bivariate_legendre(hyper_params[0], basis_dim).numpy(), 4)

array([[1., 2., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 2., 3., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 2., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 3.]])

In [264]:
# condition number using beta
n = len(thetastar[0])
sum_path = np.zeros((basis_dim * n, basis_dim * n))
for i in range(len(hyper_params)):
    theta_i = model(hyper_params[i]).detach().numpy()
    phi = phi_matrix(bivariate_legendre(hyper_params[i], basis_dim).numpy(), n)
    sum_path += phi.T @ (hessian(theta_i, mu) + 2 * hyper_params[i][0] * const.T @ const) @ phi
sim_exp_hessian_beta = sum_path/len(thetastar)
sim_exp_hessian_beta

array([[222.34643858,  32.53252635,  86.63309833, 109.26348108,
         15.98651301,  42.57234003, 116.99199356,  17.11867229,
         45.58408711, 115.80776579,  16.94519331,  45.12260186,
         94.69568161,  13.85245816,  36.89537078, 108.91316256,
         15.93519442,  42.43582336,  89.59493887,  13.10524409,
         34.90764713,  78.74387216,  11.51565799,  30.67906257,
        112.51534616,  16.46288272,  43.83956903],
       [ 32.53252635, 176.61331388,  -4.98645551,  15.98651301,
         86.7897538 ,  -2.45077876,  17.11867229,  92.92849773,
         -2.62251886,  16.94519331,  91.98786769,  -2.5962034 ,
         13.85245816,  75.21857616,  -2.1270586 ,  15.93519442,
         86.51149641,  -2.44299412,  13.10524409,  71.16706513,
         -2.01371181,  11.51565799,  62.54808183,  -1.77258348,
         16.46288272,  89.37270453,  -2.52304049],
       [ 86.63309833,  -4.98645551, 242.67997087,  42.57234003,
         -2.45077876, 119.25567035,  45.58408711,  -2.62251886,
  

In [265]:
eigs_hessian_beta = np.linalg.eigvals(sim_exp_hessian_beta)
eigs_hessian_beta

array([1220.24019589,  723.3166754 ,  487.96380448,  132.531383  ,
         87.71352811,   78.55989296,   52.99805747,   52.00065081,
         38.51263117,   35.08094686,   32.73355104,   28.93613156,
         27.0126328 ,   23.47978924,   22.84110888,    8.32862076,
         20.80119291,   19.41622814,    9.39948973,   17.16577463,
         16.025928  ,   15.40954635,   10.81196228,   11.58090969,
         13.93216895,   13.09908728,   12.34477237])

In [266]:
# condition number for Hessian(\theta)
max(eigs_hessian_beta)/min(eigs_hessian_beta)

np.float64(146.511677098368)

In [ ]:
# .01: 131
# 1: 138
# 10: 146